In [1]:
import pyspark
sc = pyspark.SparkContext("local", "sample")

22/04/06 11:36:34 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/06 11:36:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/06 11:36:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/06 11:36:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/06 11:36:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
lit=sc.wholeTextFiles("lit")
ff=sc.wholeTextFiles("fanfic")

In [ ]:
#removing \r characters, replace \n with a space, removing ASCII characters
lit=lit.map(lambda x: (x[0], x[1].replace('\r', '')))\
    .map(lambda x: (x[0], x[1].replace('\n', ' ')))
    #.map(lambda x: (x[0], x[1].replace('\\', '')))\
    #.map(lambda x: (x[0], x[1].replace('|', '')))\
    #.map(lambda x: (x[0], x[1].replace('/', '')))\
    #.map(lambda x: (x[0], x[1].replace('_', '')))\

ff=ff.map(lambda x: (x[0], x[1].replace('\r', '')))\
    .map(lambda x: (x[0], x[1].replace('\n', ' ')))
    #.map(lambda x: (x[0], x[1].replace('\\', '')))\
    #.map(lambda x: (x[0], x[1].replace('|', '')))\
    #.map(lambda x: (x[0], x[1].replace('/', '')))\
    #.map(lambda x: (x[0], x[1].replace('_', '')))\

In [26]:
erewhon1 = sc.wholeTextFiles("erewhon.txt")

In [27]:
erewhon2_quotes = erewhon1.flatMap(lambda x: x[1].split('”'))

In [28]:
def to_tuple(x):
    return (x, 1)
erewhon3_quotes = erewhon2_quotes.map(to_tuple)

In [29]:
erewhon4_quotes = erewhon3_quotes.map(lambda x: x[1]).sum()

In [32]:
erewhon_cquotes = erewhon4_quotes - 1
erewhon_cquotes

18

In [35]:
erewhon2_periods = erewhon1.flatMap(lambda x: x[1].split('.'))

In [36]:
erewhon3_periods = erewhon2_periods.map(to_tuple)

In [37]:
erewhon4_periods = erewhon3_periods.map(lambda x: x[1]).sum()

In [56]:
erewhon_periods = erewhon4_periods - 1

In [50]:
import re
# erewhon2_puncts = erewhon1.flatMap(lambda x: x[1].split(',|-|:|;'))
erewhon2_puncts = erewhon1.flatMap(lambda x: re.split(',|-|:|;', x[1]))

In [51]:
erewhon3_puncts = erewhon2_puncts.map(to_tuple)

In [52]:
erewhon4_puncts = erewhon3_puncts.map(lambda x: x[1]).sum()

In [53]:
erewhon4_puncts

727

In [41]:
erewhon2_words = erewhon1.flatMap(lambda x: x[1].split(' '))

In [42]:
erewhon3_words = erewhon2_words.map(to_tuple)

In [43]:
erewhon4_words = erewhon3_words.map(lambda x: x[1]).sum()

In [44]:
erewhon4_words

9282

In [46]:
erewhon_cquotes/erewhon4_words

0.0019392372333548805

In [57]:
erewhon4_words/erewhon_periods

41.810810810810814

In [58]:
erewhon4_puncts/erewhon_periods

3.274774774774775